#### Imports

In [1]:
from pyvis.network import Network
from faker import Faker
import random
from Graph import Graph
from typing import Optional 

# Social Network class

This class implements from the graph class but implements some more social network specific methods.

1. _get_total_likes()_
2. _get_total_comments()_
3. _follower_count()_

## Engament Rate
These 3 methods are used to calculate the engament rate based on this formula:

$$
\text{Engagement Rate} = \frac{{\text{Total Likes} + \text{Total Comments}}}{{\text{Followers}}} \times 100
$$

## Member infuelnce rate

This metric is used to calculate the influence of a member _A_ to a member _B_.

$$
\text{A to B influence} = \frac{{\text{Likes A to B} + \text{Comments A to B}}}{{\text{engagement rate A}}}
$$

This formula need 2 methods to be calculated:

1. _likes_from_user_count()_
2. _comments_from_user_count()_


###  Most infuential path
To find the most infuelential path a modified vertsion of Dijkstra's algorithm is used 




In [2]:
class X2(Graph):
    def __init__(self, directed=True):
        super().__init__(directed=directed)
        
    def get_total_likes(self, user: 'Graph.Vertex') -> int:
        edges: 'Graph.Edge' = self.incident_edges(user, outgoing=False)
        return sum(edge.element['likes'] for edge in edges)


    def get_total_comments(self, user: 'Graph.Vertex') -> int:
        edges: 'Graph.Edge' = self.incident_edges(user, outgoing=False)
        return sum(edge.element['comments'] for edge in edges)

    def follower_count(self, user: 'Graph.Vertex') -> int:
        return self.degree(user, outgoing=False)
    
    def engagement_rate(self, user: 'Graph.Vertex') -> float:
        return ((self.get_total_likes(user) + self.get_total_comments(user)) / self.follower_count(user)) * 100

    def likes_from_user_count(self, source_user: 'Graph.Vertex', destination_user: 'Graph.Vertex') -> int:

        edges = self.incident_edges(destination_user, outgoing=False)

        for edge in edges:
            if edge._origin == source_user:
                return edge.element['likes']


    def comments_from_user_count(self, source_user: 'Graph.Vertex', destination_user: 'Graph.Vertex') -> int:

        edges = self.incident_edges(destination_user, outgoing=False)
        
        for edge in edges:
            if edge._origin == source_user:
                return edge.element['comments']
    
    def influence_rate(self, source_user: 'Graph.Vertex', destination_user: 'Graph.Vertex') -> float:
        return ((self.likes_from_user_count(source_user, destination_user) \
                + self.comments_from_user_count(source_user, destination_user)) / self.engagement_rate(source_user) )
    


    def dijkstra_most_infuential(self, source: 'Graph.Vertex', destination: 'Graph.Vertex') -> tuple[float, list]:
        # Initialize influence scores and paths with zero for all vertices except the source
        influences: dict = {vertex: 0 for vertex in self.vertices()}
        
        paths: dict = {vertex: [] for vertex in self._outgoing}
        paths[source] = [source]
        
        pq: list[tuple[int, 'Graph.Vertex']] = [(0, source)]  # List of tuples (influence score, vertex)
        visited: set = set()

        while pq:
            pq.sort(key=lambda x: x[0])
            current_influence, current_vertex = pq.pop()

            if current_vertex in visited:
                continue

            # Mark the current vertex as visited
            visited.add(current_vertex)

            if current_vertex == destination:
                return current_influence, paths[destination]

            # Check neighbors and update influence scores and paths
            for edge in self.incident_edges(current_vertex):
                neighbor = edge.opposite(current_vertex)
                new_influence = influences[current_vertex] + self.influence_rate(current_vertex, neighbor)
                if new_influence > influences[neighbor]:
                    influences[neighbor] = new_influence
                    paths[neighbor] = paths[current_vertex] + [neighbor]
                    pq.append((new_influence, neighbor))

        # If the destination vertex is not reachable, return None
        return float('-inf'), None

# Generating Vertices and edges

In [3]:
x = X2()
Faker.seed(42)
fake = Faker()
random.seed(42)

To modify the number of edges and vertices change the number in the for loop

In [4]:
num_of_vertices = 100
num_of_edges = 800

In [5]:
vertices = [fake.name() for _ in range(num_of_vertices)]
edges = []
for _ in range(num_of_edges):
    # Ensure that the two vertices for an edge are different
    u_vertex, v_vertex = random.sample(vertices, 2)
    edge_data = {"likes": random.randint(1000, 10000), "comments": random.randint(1000, 10000)}
    edges.append([u_vertex, v_vertex, edge_data])

In [6]:
vertex_objects = {}  # Dictionary to store mapping of vertex element to Vertex object
    
for v in vertices:
    vertex_object = x.insert_vertex(v)
    vertex_objects[v] = vertex_object
    

for edge in edges:
    x.insert_edge(edge[0], edge[1], edge[2])


# Selecting 2 sample users for bfs and dijkstra

In [7]:
test_users = random.sample(vertices, 2)

#### This code prints the result of bfs and dijkstra, for the graph scroll down

In [8]:
print(x.bfs(vertex_objects[test_users[0]], vertex_objects[test_users[1]])) 
influence_score, path= x.dijkstra_most_infuential(vertex_objects[test_users[0]], vertex_objects[test_users[1]])
print(influence_score, path, len(path))

(3, ['Jeffrey Chavez', 'Cristian Santos', 'Sherri Baker', 'Angela Lopez'])
1.0275687738261552 ['Jeffrey Chavez', 'Lisa Jackson', 'Megan Young', 'Tommy Walter', 'Amy Silva', 'Danny Morgan', 'Joshua Blair', 'Ryan Munoz', 'Cassandra Gaines', 'John Whitehead', 'Connie Lawrence', 'Philip Cannon', 'Shane Henderson', 'Sherri Baker', 'Brent Jordan', 'Holly Wood', 'Deborah Figueroa', 'Margaret Hawkins DDS', 'Christina Walters', 'Paula Moreno', 'Kelly Donovan', 'Rebecca Henderson', 'Matthew Foster', 'Stephanie Ross', 'Judy Baker', 'Eric Carney', 'Whitney Peters', 'Fred Smith', 'Patty Perez', 'Tricia Valencia', 'Zachary Ferrell', 'Shannon Jones', 'Dylan Miller', 'Jessica Smith', 'Stephen Mckee', 'William Baker', 'Matthew Mcmillan', 'Anthony Rodriguez', 'Sherry Decker', 'Jordan Henderson', 'Carol Tucker', 'John Pierce', 'Lisa Barnes', 'Mike Allen', 'Aaron Wise', 'Timothy Duncan', 'Mark Perez', 'Aimee Montoya', 'Derek Zuniga', 'Denise Jacobs', 'Sarah Moore', 'Michele Williams', 'Lauren Daniels', 'S

# Visualization of the entire network

In [9]:
# Create a Pyvis network instance
net = Network(notebook = True, cdn_resources = "remote",
            bgcolor = "#222222",
            font_color = "white",
            height = "1000",
            width = "100%",
            directed = True,
            select_menu=True
)

net.set_options("""
const options = {
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -16850,
      "centralGravity": 0,
      "springLength": 190,
      "springConstant": 0,
      "damping": 0.1
    },
    "minVelocity": 0.75
  }
}
""")

# Add nodes (vertices) to the network
for vertex in x.vertices():
    net.add_node(vertex.element, title=str(vertex.element))

# Add edges to the network
for edge in x.edges():
    net.add_edge(edge._origin.element, edge._destination.element,
                    title=f"Likes: {edge.element['likes']} | Comments: {edge.element['comments']}")

# Visualize the network
net.show("graph.html")

graph.html


# Visualization of most infuential path using modified Dijkstra's algorithm from node A to B

In [10]:
distance, path = x.dijkstra_most_infuential(vertex_objects[test_users[0]], vertex_objects[test_users[1]])
# print(f'distance: {distance}\npath taken: {path}\nnumber of steps {len(path)}')

net = Network(notebook = True, cdn_resources = "remote",
            bgcolor = "#222222",
            font_color = "white",
            directed = True,
            height = "600",
            width = "100%",
)
# Print the type of each element in the path
for vertex in path:
    net.add_node(vertex.element, title=str(vertex.element))

for i in range(len(path) - 1):
    edge = x.get_edge(path[i], path[i+1])
    net.add_edge(path[i].element, path[i+1].element, title=f"Likes: {edge.element['likes']} | Comments: {edge.element['comments']}")

net.show("dijksta.html")

dijksta.html


# Visualization of shortest path using bfs 

In [11]:
distance, path = x.bfs(vertex_objects[test_users[0]], vertex_objects[test_users[1]])
# print(f'distance: {distance} path taken: {path}')

net = Network(notebook = True, cdn_resources = "remote",
            bgcolor = "#222222",
            font_color = "white",
            height = "600",
            width = "100%",
            directed = True
)
# Print the type of each element in the path
for vertex in path:
    net.add_node(vertex.element, title=str(vertex.element))

for i in range(len(path) - 1):
    edge = x.get_edge(path[i], path[i+1])
    net.add_edge(path[i].element, path[i+1].element, title=f"Likes: {edge.element['likes']} | Comments: {edge.element['comments']}")

net.show("bfs.html")

bfs.html
